In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
pd.set_option('max_columns', 120)
pd.set_option('max_colwidth', 5000)
%matplotlib inline

In [2]:
tn_police = pd.read_csv('../data/tn_nashville_2019_08_13.csv', low_memory=False)
tn_police.shape

(3092351, 42)

In [3]:
tn_police = tn_police.dropna(thresh=len(tn_police)/2, axis=1) ## Remove columns that have many missing values
tn_police.shape

(3092351, 37)

In [4]:
tn_police.head(2)

,raw_row_number,date,time,location,lat,lng,precinct,reporting_area,zone,subject_age,subject_race,subject_sex,officer_id_hash,type,violation,arrest_made,citation_issued,warning_issued,outcome,frisk_performed,search_conducted,search_person,search_vehicle,reason_for_stop,vehicle_registration_state,raw_verbal_warning_issued,raw_written_warning_issued,raw_traffic_citation_issued,raw_misd_state_citation_issued,raw_suspect_ethnicity,raw_driver_searched,raw_passenger_searched,raw_search_consent,raw_search_arrest,raw_search_warrant,raw_search_inventory,raw_search_plain_view
0,232947,2010-10-10,NaN,"DOMINICAN DR & ROSA L PARKS BLVD, NASHVILLE, TN, 37208",36.187925,-86.798519,6,4403.0,611,27.0,black,male,80ed1b32eb,vehicular,investigative stop,False,False,True,warning,False,False,False,False,investigative stop,TN,True,False,False,NaN,N,False,False,False,False,False,False,False
1,237161,2010-10-10,10:00:00,"1122 LEBANON PIKE, NASHVILLE, TN, 37210",36.155521,-86.735902,5,9035.0,513,18.0,white,male,a983204b21,vehicular,moving traffic violation,False,True,False,citation,False,False,False,False,moving traffic violation,TN,False,False,True,NaN,N,False,False,False,False,False,False,False


In [5]:
tn_police['type'].value_counts()

vehicular    3092351
Name: type, dtype: int64

## Feature Engineering

In [6]:
# Change the data format and get the year, month, day and hour 
# Get the Zip 
tn_police['date'] = pd.to_datetime(tn_police['date'])
tn_police['year'], tn_police['month'], tn_police['day'] = tn_police['date'].dt.year, tn_police['date'].dt.month, tn_police['date'].dt.day
tn_police['time'] = tn_police['time'].str[0:2]
# tn_police['zip'] = tn_police['location'].str.extract(r'(\d{5}\-?\d{0,4})')


drop_columns = ['raw_row_number','date','location', 'lat','lng','reporting_area','officer_id_hash','arrest_made', 'citation_issued', 'warning_issued',
                'search_conducted','vehicle_registration_state','reason_for_stop',
               'raw_verbal_warning_issued', 'raw_written_warning_issued', 'raw_traffic_citation_issued',
               'raw_misd_state_citation_issued', 'raw_suspect_ethnicity','raw_driver_searched', 'raw_passenger_searched',
               'raw_search_consent', 'raw_search_arrest', 'raw_search_warrant', 'raw_search_inventory', 'raw_search_plain_view']
tn_police = tn_police.drop(drop_columns, axis=1)

tn_police.shape

(3092351, 15)

In [7]:
tn_police.head(2)

,time,precinct,zone,subject_age,subject_race,subject_sex,type,violation,outcome,frisk_performed,search_person,search_vehicle,year,month,day
0,NaN,6,611,27.0,black,male,vehicular,investigative stop,warning,False,False,False,2010,10,10
1,10,5,513,18.0,white,male,vehicular,moving traffic violation,citation,False,False,False,2010,10,10


In [8]:
# Change the Categorical Variables 

# Drop all nan
tn_police = tn_police.dropna()

tn_police['subject_age'] = tn_police['subject_age'].astype('int')
tn_police['time'] = tn_police['time'].astype('float')

## Encode the Target Variable 
# 1 --> Arrested  
# 0 --> Not Arrested  
outcome = {
    'citation':1,
    'arrest':1,
    'warning':0 
}

tn_police['target'] = tn_police['outcome'].replace(outcome)
tn_police = tn_police.drop('outcome', axis=1)

In [9]:
tn_police.head(10)

,time,precinct,zone,subject_age,subject_race,subject_sex,type,violation,frisk_performed,search_person,search_vehicle,year,month,day,target
1,10.0,5,513,18,white,male,vehicular,moving traffic violation,False,False,False,2010,10,10,1
2,10.0,1,121,52,white,male,vehicular,vehicle equipment violation,False,False,False,2010,10,10,0
3,22.0,3,325,25,white,male,vehicular,registration,False,False,False,2010,10,10,0
5,1.0,7,727,26,white,female,vehicular,moving traffic violation,False,False,False,2010,10,10,0
7,10.0,7,721,33,white,male,vehicular,seatbelt violation,False,False,False,2010,10,10,0
9,10.0,8,831,49,black,male,vehicular,seatbelt violation,False,False,False,2010,10,10,1
10,10.0,2,221,18,hispanic,female,vehicular,moving traffic violation,False,False,False,2010,10,10,1
11,10.0,3,331,31,white,female,vehicular,moving traffic violation,False,False,False,2010,10,10,0
13,22.0,3,315,35,white,male,vehicular,safety violation,False,False,False,2010,10,10,0
14,10.0,5,513,27,white,male,vehicular,moving traffic violation,False,False,False,2010,10,10,1


In [11]:
tn_police.shape

(2677735, 15)

In [12]:
## Other Feature Columns Encoding 

le = LabelEncoder()
#convert the categorical columns into numeric
tn_police['subject_sex'] = le.fit_transform(tn_police['subject_sex'])
tn_police['frisk_performed'] = le.fit_transform(tn_police['frisk_performed'])
tn_police['search_person'] = le.fit_transform(tn_police['search_person'])
tn_police['search_vehicle'] = le.fit_transform(tn_police['search_vehicle'])
tn_police['type'] = le.fit_transform(tn_police['type'])


#display the initial records

nominal_columns = ["subject_race", "violation",'precinct','zone']
new_columns = pd.get_dummies(tn_police[nominal_columns])
tn_police = pd.concat([tn_police, new_columns], axis=1)
tn_police = tn_police.drop(nominal_columns, axis=1)
tn_police.head()


,time,subject_age,subject_sex,type,frisk_performed,search_person,search_vehicle,year,month,day,target,subject_race_asian/pacific islander,subject_race_black,subject_race_hispanic,subject_race_other,subject_race_unknown,subject_race_white,violation_child restraint,violation_investigative stop,violation_moving traffic violation,violation_parking violation,violation_registration,violation_safety violation,violation_seatbelt violation,violation_vehicle equipment violation,precinct_1,precinct_2,precinct_3,precinct_4,precinct_5,precinct_6,precinct_7,precinct_8,precinct_U,zone_111,zone_113,zone_115,zone_117,zone_12,zone_121,zone_123,zone_125,zone_13,zone_211,zone_213,zone_215,zone_217,zone_22,zone_221,zone_223,zone_225,zone_227,zone_31,zone_311,zone_313,zone_315,zone_317,zone_32,zone_321,zone_323,zone_325,zone_33,zone_331,zone_333,zone_335,zone_41,zone_411,zone_413,zone_413N,zone_415,zone_421,zone_423,zone_425,zone_51,zone_511,zone_513,zone_515,zone_515K,zone_517,zone_52,zone_521,zone_523,zone_525,zone_53,zone_531,zone_533,zone_535,zone_61,zone_611,zone_613,zone_615,zone_617,zone_62,zone_621,zone_623,zone_625,zone_627,zone_63,zone_711,zone_713,zone_715,zone_721,zone_723,zone_725,zone_727,zone_811,zone_813,zone_815,zone_821,zone_823,zone_825,zone_831,zone_833,zone_835,zone_UNK
1,10.0,18,1,0,0,0,0,2010,10,10,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10.0,52,1,0,0,0,0,2010,10,10,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,22.0,25,1,0,0,0,0,2010,10,10,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1.0,26,0,0,0,0,0,2010,10,10,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
7,10.0,33,1,0,0,0,0,2010,10,10,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [13]:
tn_police.shape

(2677735, 115)